# Titanic: Machine Learning From Disaster
### by Sung Anh and Abdul Saleh

<hr>
## Introduction
In this project we use random forest and xgboost machine learning algorithms to predict who survived the sinking of the RMS Titanic. In as much as this is a learning experience, our purpose in this project is not just to accurately predict who survived and who didn't, but we also focus a lot on using Also, it is worth noting that the Titanic dataset kindly provided by Kaggle and used in this project is one of the most commonly analyzed datasets in all of machine learning. So in our own analysis we rely on the insightful remarks and conclusions introduced by the wonderful data science community. We are basically standing on the shoulders of giants drawing on their experiences and learning from their mistakes.

<hr>
## Outline
1. Understanding the Problem
    - Machine learning, and CS in general, is about problem solving so we begin by understanding the problem
2. Exploratory Data Analysis
    - We take a good look at the data, mess around with it, and decide which approach to take

In [1]:
print("hello,world")

hello,world


This is another test